# Structure
Define the objects of parameters.

In [3]:
Base.@kwdef struct StrucPar
    r::Float64 ## interest rate
    eta::Float64 ## elasticity in matching function
    beta::Float64 ## workers' bargaining power
    delta::Float64 ## job destruction rate at steady state
    rho::Float64 ## workers' exit rate
    lambda::Float64 ## U to O constant rate
    chi::Float64 ## O to U constant rate
    Phi_star::Float64 ## measure of entrant workers at ss
    Psi_star::Float64 ## measure of entrant firms at ss
    q_star::Float64 ## firms' job-filling rate at ss                
    theta_q_star::Float64 ## workers' job-finding rate at ss
    theta_star::Float64 ## market tightness at ss
    
    d_star::Float64 ## measure of workers at ss   
    l_star::Float64 ## measure of workers in LF at ss          
    m_star::Float64 ## measure of jobs at ss 
    n_star::Float64 ## measure of firms at ss 

    gamma::Float64 
    nuF::Float64 ## participation response parameter
    nuPhi::Float64 ## semi-elasticity of Phi
    nuPsi::Float64 ## semi-elasticity of Psi
           
    Y_star::Float64 ## ss productivity
    b::Float64 ## home production
    c::Float64 ## search cost
    Ztilde::Float64 ## workers' value from exiting
    Vtilde::Float64 ## firms' value from exiting
    P_star::Float64
    S_star::Float64 ## job surplus at ss
    Sw_star::Float64 ## workers' job surplus at ss
    Sf_star::Float64 ## firms' job surplus at ss
    w_star::Float64 ## wage at ss
    H_star::Float64 ## housing cost at ss
    K_star::Float64 ## business cost at ss
    U_star::Float64 ## value of unemployment at ss
    V_star::Float64 ## value of vacancy at ss
    Z_star::Float64
    
    
    ## agglomeration elasticities
    eHd::Float64 
    eHn::Float64
    eKd::Float64
    eKn::Float64
    eYd::Float64
    eYn::Float64
    
    ## shock
    a::Float64 ## persistence: 0= no persistence
    delta_jump::Float64 ## initial job destruction rate jump(%)
    A_drop::Float64 ## initial TFP drop (%)
    
    ##
    version::Symbol ##version: :newb, :fullpar, :rigidwage, :rigidrent
end

Base.@kwdef struct SimPar
    M::Integer ## number of points in time
    mu::Float64 ## scale of transformation [0,1] into [0,Inf], large mu -> time points closer to 0
    h::Float64 ## initial step in line search algorithm
    maxiter::Integer ## number of iterations in the Newton-type algorithm
    maxerr::Float64 ## maximum error in Newton-type algorithm
    data::DataFrame ## empirical moments to match
end


 
Base.@kwdef struct PolicyPar   
    gw::Float64 ## subsidy to workers
    gf::Float64 ## subsidy to firms
    subdur::Float64 ## duration of subsidy
end

PolicyPar

# I) The baseline model
An equilibrium is defined as the solution of these equations:
\begin{align}
    & \dot{d}_t=\Phi(U_t-G(P_t))-\rho d_t \\
	&\dot{l}_t=F(P_t)\Phi(U_t-G(P_t))+(\chi+\gamma F(P_t))(d_t-l_t) - \gamma(1-F(P_t)) (l_t-m_t) - (\rho+\lambda)l_t \\
	&\dot{n}_t=\Psi(V_t)-\tilde\delta_t n_t \\
	& \dot{m}_t= q(\theta_t)(n_{t}-m_{t})-(\rho+\lambda+\tilde \delta_t) m_{t}\\
    &  \theta_t=\frac{n_{t}-m_{t}}{l_{t}-m_{t}}\\
    &[r+\rho+\lambda+\tilde \delta_t+\beta\theta_tq(\theta_t)+(1-\beta)q(\theta_t)]S_t=\tilde A_t y_t-b+c+\gamma G(P_t)+\dot{S}_t\\
    & (r+\rho+\lambda+\chi+\gamma) P_t= -c+\beta\theta_t q(\theta_t) S_t+\dot{P}_t\\
    & (r+\rho) U_t=\rho \tilde Z + b-h_{t}-c-\lambda P_t-\gamma G(P_t)+\beta\theta_t q(\theta_t) S_t+\dot{U}_t\\
	& (r+\tilde\delta_t) V_t=\tilde \delta_t \tilde V-k_t+(1-\beta) q(\theta_t) S_t+\dot{V}_t
\end{align}

We first propose a test for a steady-state equilibrium.

In [ ]:
# test if the steady-state as defined by the model is well-defined
function test_steadystate(p::StrucPar) 
    out = zeros(9)
    out[1] = (p.Phi_star - p.rho * p.d_star ) / p.d_star
    F = F_fun(p.P_star,p.nuF)
    out[2] = ( F *p.Phi_star + (p.chi+p.gamma*F)*(p.d_star-p.l_star) - p.gamma*(1-F)*(p.l_star-p.m_star)- (p.rho + p.lambda) * p.l_star ) / p.l_star
    out[3] = (p.Psi_star - p.delta * p.n_star) / p.n_star
    out[4] = (p.theta_q_star * (p.l_star-p.m_star) - (p.rho + p.lambda + p.delta) * p.m_star ) / p.m_star 
    out[5] = (p.q_star * (p.n_star-p.m_star) - (p.rho+ p.lambda +p.delta) * p.m_star ) / p.m_star 
    G = G_fun(p.P_star,p.nuF)
    out[6] = ( (p.r+p.rho) * p.U_star - p.rho*p.Ztilde - p.b + p.H_star +p.c - p.beta*p.theta_star*p.q_star*p.S_star +p.lambda*p.P_star + p.gamma*G) / p.U_star
    out[7] = ( (p.r+p.delta)*p.V_star - p.delta*p.Vtilde + p.K_star - (1-p.beta) * p.q_star *p.S_star) / p.V_star 
    out[8] = ((p.r +p.rho+p.lambda+ p.delta + p.beta*p.theta_q_star + (1-p.beta)*p.q_star) * p.S_star - 
        p.Y_star + p.b -p.c -p.gamma*G) / p.S_star 
    out[9] = ( (p.r+p.rho+p.lambda+p.chi+p.gamma) * p.P_star +p.c - p.beta*p.theta_star*p.q_star*p.S_star) / p.P_star  
    println("TEST if the parameters well define a steady state. The following numbers should all be close to 0.")
    println(out)
    p.S_star <0 ? print("/!\\ S_star negative") : ()
    return()
end

For the participation response, we have:
\begin{align}
    & F(P)=1-\exp(-P/\nu_F )\\
    & G(P)=\nu_F\left(1-\exp(-P/\nu_F )\right)
\end{align}


We also assume some functions are linear or log-linear:
\begin{align}
    & \Phi(Z)=\Phi(Z^*)\exp(\nu_\Phi(Z-Z^*))\\
     & \Psi(V)=\Psi(V^*)\exp(\nu_\Psi(V-V^*))\\
    & q(\theta) = q(\theta^*)\left(\frac{\theta}{\theta^*}\right)^{\eta-1}\\
    & \mathcal{H}(d,n)=\mathcal{H}(d^*,n^*)\left(\frac{d}{d^*}\right)^{\epsilon_d^H}\left(\frac{n}{n^*}\right)^{\epsilon_n^H}\\
    & \mathcal{K}(d,n)=\mathcal{K}(d^*,n^*)\left(\frac{d}{d^*}\right)^{\epsilon_d^K}\left(\frac{n}{n^*}\right)^{\epsilon_n^K}\\
    & \mathcal{Y}(d,n)=\mathcal{Y}(d^*,n^*)\left(\frac{d}{d^*}\right)^{\epsilon_d^Y}\left(\frac{n}{n^*}\right)^{\epsilon_n^Y}
\end{align}

In [ ]:
F_fun(P::Float64, nuF::Float64) = 1-exp(-P / nuF)
FD_fun(P::Float64, nuF::Float64) = exp(-P / nuF)  / nuF
G_fun(P::Float64, nuF::Float64) = F_fun(P, nuF) * nuF
GD_fun(P::Float64, nuF::Float64) = exp(-P / nuF)

Phi_fun(Z::Float64,p::StrucPar) = p.Phi_star * exp( p.nuPhi * (Z-p.Z_star) )
PhiD_fun(Z::Float64,p::StrucPar) = p.nuPhi * Phi_fun(Z,p)

Psi_fun(V::Float64,p::StrucPar) = p.Psi_star *exp( p.nuPsi * (V-p.V_star) )
PsiD_fun(V::Float64,p::StrucPar) = p.nuPsi * Psi_fun(V,p)

q_fun(theta::Float64,p::StrucPar) = p.q_star * (theta/p.theta_star)^(p.eta-1)

#### rename Hl etc

H_fun(d::Float64, n::Float64, p::StrucPar) = p.H_star * (d/p.d_star)^p.eHd * (n/p.n_star)^p.eHn
Hd_fun(d::Float64, n::Float64, p::StrucPar) = p.eHd  * H_fun(d,n,p) / d
Hn_fun(d::Float64, n::Float64, p::StrucPar) = p.eHn  * H_fun(d,n,p) / n
K_fun(d::Float64, n::Float64, p::StrucPar) = p.K_star * (d/p.d_star)^p.eKd * (n/p.n_star)^p.eKn
Kd_fun(d::Float64, n::Float64, p::StrucPar) = p.eKd  * K_fun(d,n,p) / d
Kn_fun(d::Float64, n::Float64, p::StrucPar) = p.eKn  * K_fun(d,n,p) / n

Y_fun(d::Float64, n::Float64, p::StrucPar) = p.Y_star * (d/p.d_star)^p.eYd * (n/p.n_star)^p.eYn
Yd_fun(d::Float64, n::Float64, p::StrucPar) = p.eYd  * Y_fun(d,n,p) / d
Yn_fun(d::Float64, n::Float64, p::StrucPar) = p.eYn  * Y_fun(d,n,p) / n

The shock shifts $A^*$ and $\delta^*$ to $A_0$ and $\delta_0$ with a persistence $a$.
\begin{align}
    & \tilde\delta_t=\delta+(\delta_0-\delta) \exp(-t/a)\\
    & \ln \tilde A_t  =  \ln A_0 \exp(-t/a)
\end{align}

In [ ]:
### job separation shock
# delta0 =(1+delta_jump) * delta
deltatilde_fun(t::Float64, p::StrucPar) = p.delta * (1 + p.delta_jump * exp(- t / p.a))

## TFP shock
# A0 = (1-A_drop) * A, with A=1
Atilde_fun(t::Float64, p::StrucPar) = exp( log(1-p.A_drop) * exp(- t / p.a) )

To create or modify a StrucPar

In [ ]:
# build the same model with other agglo forces
function StrucPar_agglo(eHd, eHn, eKd, eKn, eYd, eYn, p::StrucPar)
    StrucPar(r=p.r,eta=p.eta,beta=p.beta,delta=p.delta,rho=p.rho,Phi_star=p.Phi_star,q_star=p.q_star, 
        chi=p.chi, lambda=p.lambda, gamma=p.gamma,
        theta_q_star=p.theta_q_star, theta_star = p.theta_star,
        d_star=p.d_star, l_star=p.l_star,m_star =p.m_star, n_star=p.n_star,Y_star=p.Y_star,
        Psi_star= p.Psi_star, c=p.c,
        nuPhi=p.nuPhi , nuPsi=p.nuPsi, nuF=p.nuF,
        b=p.b, Ztilde=p.Ztilde, Vtilde=p.Vtilde,
        S_star = p.S_star, Sw_star=p.Sw_star, Sf_star=p.Sf_star,
        w_star = p.w_star,
        H_star = p.H_star, K_star = p.K_star,
        U_star = p.U_star, V_star = p.V_star, P_star=p.P_star, Z_star=p.Z_star,
        eHd=eHd, eHn=eHn, eKd=eKd, eKn=eKn, eYd=eYd, eYn=eYn, 
        a=p.a, delta_jump=p.delta_jump, A_drop=p.A_drop, version=p.version)
end

function StrucPar_migr(nuPhi, nuPsi, p::StrucPar)
    StrucPar(r=p.r,eta=p.eta,beta=p.beta,delta=p.delta,rho=p.rho,Phi_star=p.Phi_star,q_star=p.q_star, 
        chi=p.chi, lambda=p.lambda, gamma=p.gamma,
        theta_q_star=p.theta_q_star, theta_star = p.theta_star,
        d_star=p.d_star, l_star=p.l_star,m_star =p.m_star, n_star=p.n_star,Y_star=p.Y_star,
        Psi_star= p.Psi_star, c=p.c,
        nuPhi=nuPhi , nuPsi=nuPsi, nuF=p.nuF,
        b=p.b, Ztilde=p.Ztilde, Vtilde=p.Vtilde,
        S_star = p.S_star, Sw_star=p.Sw_star, Sf_star=p.Sf_star,
        w_star = p.w_star,
        H_star = p.H_star, K_star = p.K_star,
        U_star = p.U_star, V_star = p.V_star, P_star=p.P_star, Z_star=p.Z_star,
        eHd=p.eHd, eHn=p.eHn, eKd=p.eKd, eKn=p.eKn, eYd=p.eYd, eYn=p.eYn, 
        a=p.a, delta_jump=p.delta_jump, A_drop=p.A_drop, version=p.version)
end
